In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV 
import joblib 
import os 

df_obesity = pd.read_csv("C:\\Users\\Igor\\Documents\\GitHub\\Tech_Challenge_Fase_4_Data_Analitycs\\data\\Obesity.csv")


# Passos de exploração baixo estão descrito no Exploracao.ipynb
novos_nomes = {
    'Gender': 'Gênero',
    'Age': 'Idade',
    'Height': 'Altura',
    'Weight': 'Peso',
    'family_history': 'Histórico_Familiar_Obesidade',
    'FAVC': 'Frequencia_Consumo_Alimento_Calorico',
    'FCVC': 'Frequencia_Consumo_Vegetais',
    'NCP': 'Numero_Refeicoes_Principais',
    'CAEC': 'Consumo_Alimento_Entre_Refeicoes',
    'SMOKE': 'Fumante',
    'CH2O': 'Consumo_Agua',
    'SCC': 'Monitoramento_Calorico',
    'FAF': 'Frequencia_Atividade_Fisica',
    'TUE': 'Tempo_Uso_Tecnologia',
    'CALC': 'Consumo_Alcool',
    'MTRANS': 'Meio_Transporte',
    'Obesity': 'Status_Obesidade'  
}

df_obesity = df_obesity.rename(columns=novos_nomes)

df_obesity['Idade'] = df_obesity['Idade'].astype(int)

colunas_float_arredondar = [
    'Frequencia_Consumo_Vegetais',
    'Numero_Refeicoes_Principais',
    'Consumo_Agua',
    'Frequencia_Atividade_Fisica',
    'Tempo_Uso_Tecnologia'
]

for coluna in colunas_float_arredondar:
    df_obesity[coluna] = df_obesity[coluna].astype(int)

mapeamento_binario = {'yes': 1, 'no': 0}

colunas_binarias = [
    'Histórico_Familiar_Obesidade',
    'Frequencia_Consumo_Alimento_Calorico',
    'Fumante',
    'Monitoramento_Calorico'
]

for coluna in colunas_binarias:
    df_obesity[coluna] = df_obesity[coluna].str.lower().map(mapeamento_binario).astype(int)

# Gerado IMC, referencia
# Categorias de IMC para adultos:
# Abaixo do peso: IMC menor que 18,5 kg/m².
# Peso normal ou saudável: IMC entre 18,5 e 24,9 kg/m².
# Sobrepeso: IMC entre 25,0 e 29,9 kg/m².
# Obesidade Grau I: IMC entre 30,0 e 34,9 kg/m².
# Obesidade Grau II (severa): IMC entre 35,0 e 39,9 kg/m².
# Obesidade Grau III (mórbida): IMC igual ou maior que 40,0 kg/m². 
#Conforme a tabela, categorizamos a variavel em binaria, sendo =>30 obesidade < 30 nao obeso

df_obesity['IMC'] = (df_obesity['Peso'] / (df_obesity['Altura'] ** 2)).astype(float).round(1)

df_obesity['Targets'] = df_obesity['IMC'].apply(lambda x: 1 if x >= 30 else 0)


X = df_obesity.drop(['Status_Obesidade', 'Targets', 'IMC'], axis=1) # CARACTERISTICAS
y = df_obesity['Targets'] # TARGETS


# Divisão dos dados em treino e teste com estratificacao
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


#  Definir as Colunas
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist() 
categorical_features = X_train.select_dtypes(include='object').columns.tolist() 

#Criar os Pré-processadores
numeric_transformer = StandardScaler() # Padronizar numéricas
categorical_transformer = OneHotEncoder(handle_unknown='ignore', drop='first') # Codificar categóricas

#Combinar em um ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Manter colunas não listadas (se houver, como IDs, mas melhor remover)
)



param_grid = {
    # N de arvores
    'classifier__n_estimators': [100, 200, 300],
    # Profundidade máxima de cada arvrore
    'classifier__max_depth': [5, 10, None], 
    # minimo amostras necessárias para dividir um no
    'classifier__min_samples_split': [2, 5, 10]
}





# Pipeline com preprocessamento e o modelo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])





# Inicializa o GridSearchCV
grid_search = GridSearchCV(
    model_pipeline, 
    param_grid, 
    cv=5, 
    scoring='accuracy', 
    verbose=2, 
    n_jobs=-1 
)

print("Iniciando o treinamento e otimização de hyperparâmetros...")
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_



Iniciando o treinamento e otimização de hyperparâmetros...
Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [12]:


print("-" * 50)
print(f"Melhor Score (CV): {grid_search.best_score_:.4f}")
print(f"Melhores Params: {grid_search.best_params_}")
print("-" * 50)

# Avaliar o melhor modelo no conjunto de teste (NUNCA VISTO)
y_pred = best_model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred)

print(f"Accuracy (test): {accuracy_test:.4f}")
print("\nRelatório de Classificação no Teste:")
print(classification_report(y_test, y_pred))

if not os.path.exists('models'):
    os.makedirs('models')

# 5.2. Salva o melhor modelo (o pipeline completo)
caminho_modelo = 'models/verso_2_model_random_forest.joblib'
joblib.dump(best_model, caminho_modelo)

print(f"Modelo Random Forest (Pipeline e Pré-processamento) salvo com sucesso!")
print(f"Caminho: {caminho_modelo}")
print("="*60)

--------------------------------------------------
Melhor Score (CV): 0.9899
Melhores Params: {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 300}
--------------------------------------------------
Accuracy (test): 0.9858

Relatório de Classificação no Teste:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       228
           1       0.99      0.98      0.98       195

    accuracy                           0.99       423
   macro avg       0.99      0.99      0.99       423
weighted avg       0.99      0.99      0.99       423

Modelo Random Forest (Pipeline e Pré-processamento) salvo com sucesso!
Caminho: models/verso_2_model_random_forest.joblib


c:\Users\Igor\Documents\GitHub\Tech_Challenge_Fase_4_Data_Analitycs\venv_tc4\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [1]:
import joblib
import pandas as pd
import numpy as np
import os


CAMINHO_MODELO = r'C:\\Users\\Igor\\Documents\\GitHub\\Tech_Challenge_Fase_4_Data_Analitycs\\src\\models\\verso_2_model_random_forest.joblib'


try:
    print(f"Carregando modelo de: {CAMINHO_MODELO}...")
    modelo_carregado = joblib.load(CAMINHO_MODELO)
    print("Modelo carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em {CAMINHO_MODELO}. Certifique-se de que a pasta 'models' e o arquivo existem.")
    exit()

# ----------------------------------------------------------------------
# 2. Testar o Modelo com Novos Dados Simulados (Verificação de Integridade)
# ----------------------------------------------------------------------

# O modelo carregado (que é um Pipeline) espera os DADOS BRUTOS, 
# EXATAMENTE COM AS MESMAS COLUNAS, TIPOS E NOMES USADOS NO TREINAMENTO.

# Exemplo de NOVOS DADOS (dados de um novo paciente/pessoa)
dados_novo_paciente = {
    'Gênero': ['Female', 'Male','Male'],
    'Idade': [25, 40, 32],
    'Altura': [1.65, 1.80,1.75],
    'Peso': [60.0, 110.0,78.0],
    'Histórico_Familiar_Obesidade': [0, 1,1], # 'no' ou 'yes' codificado
    'Frequencia_Consumo_Alimento_Calorico': [0, 1,1],
    'Frequencia_Consumo_Vegetais': [2, 1,1], # (Inteiro)
    'Numero_Refeicoes_Principais': [3, 3,4], # (Inteiro)
    'Consumo_Alimento_Entre_Refeicoes': ['Sometimes', 'Frequently', 'Frequently'],
    'Fumante': [0, 0,1],
    'Consumo_Agua': [2, 1,3], # (Inteiro)
    'Monitoramento_Calorico': [0, 0,0],
    'Frequencia_Atividade_Fisica': [1, 0,0], # (Inteiro)
    'Tempo_Uso_Tecnologia': [0, 2,2], # (Inteiro)
    'Consumo_Alcool': ['no', 'Always', 'Always'],
    'Meio_Transporte': ['Public_Transportation', 'Automobile', 'Automobile']
}

# Criar um DataFrame de teste
df_novos_dados = pd.DataFrame(dados_novo_paciente)

print("\nDados de Entrada para Previsão (Novos Pacientes):")
print(df_novos_dados)
print("-" * 50)


# Fazer a Previsão
# O Pipeline cuidará automaticamente do Standard Scaling e One-Hot Encoding!
previsoes = modelo_carregado.predict(df_novos_dados)

print("\nPrevisões do Modelo:")
print(f"Paciente 1 (25 anos, 60kg): {previsoes[0]}")
print(f"Paciente 2 (40 anos, 110kg): {previsoes[1]}")
print(f"Paciente 3 (32 anos, 78kg): {previsoes[2]}")
print("-" * 50)

# Interpretação dos Resultados
mapeamento_target = {0: 'NÃO OBESO (IMC < 30)', 1: 'OBESO (IMC >= 30)'}
resultados_interpretados = [mapeamento_target[p] for p in previsoes]

print("Resultados Interpretados:")
print(f"Paciente 1: {resultados_interpretados[0]}")
print(f"Paciente 2: {resultados_interpretados[1]}")
print(f"Paciente 3: {resultados_interpretados[2]}")



Carregando modelo de: C:\\Users\\Igor\\Documents\\GitHub\\Tech_Challenge_Fase_4_Data_Analitycs\\src\\models\\verso_2_model_random_forest.joblib...
Modelo carregado com sucesso!

Dados de Entrada para Previsão (Novos Pacientes):
   Gênero  Idade  Altura   Peso  Histórico_Familiar_Obesidade  \
0  Female     25    1.65   60.0                             0   
1    Male     40    1.80  110.0                             1   
2    Male     32    1.75   78.0                             1   

   Frequencia_Consumo_Alimento_Calorico  Frequencia_Consumo_Vegetais  \
0                                     0                            2   
1                                     1                            1   
2                                     1                            1   

   Numero_Refeicoes_Principais Consumo_Alimento_Entre_Refeicoes  Fumante  \
0                            3                        Sometimes        0   
1                            3                       Frequently       

c:\Users\Igor\Documents\GitHub\Tech_Challenge_Fase_4_Data_Analitycs\venv_tc4\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
